# Part 4: Distributed Data Parallel

So far, we have used only a single GPU for training. But you may have multiple GPUs available. How do you use them? That's what we do here.

Let's take a moment to think about what's going to happen in a multi-GPU setting. In a real world setting, you may have multiple boxes (nodes) and each of it may have multiple GPUs. You have your script for training the model. It's good to imagine it this way: several Python interpreters (depending on the total number of GPUs) are going to run the exact same script in parallel, without being aware of what the others are doing. So if you have a print statement in your script, all your GPUs are going to print it. Each GPU is going to run a *process* in parallel.

You generally have a way of identifying each GPU by a unique global ID (rank). You also get the ID of the GPU within that node. PyTorch gives this by setting some environment variables such as RANK and LOCAL_RANK. All GPUs run the exact same script, they only differ by these environment variables.

Now, your task as a programmer is to ensure that you setup your script in such a way that you can run the exact same script on multiple GPUs at the same time without running into issues. You need to compute loss properly. You also need to make sure that you are passing different data to each GPU. Also, you have no control over which of the GPUs will start / finish before which other GPU. You have to assume that you cannot predict this.

You typically assign the GPU with rank = 0 as the master process. For one time tasks, such as printing the loss, you generally print only if the GPU is the master process.

Lastly, you need a way to *destroy* these processes as well for cleaning up the mess you left.

In PyTorch, you can do this using distributed data parallel.

In [ ]:
from torch.distributed import init_process_group, destroy_process_group
import os
import torch

ddp = int(os.environ.get('RANK', -1)) != -1  # if you have multiple GPUs then, this condition will be true & you want to execute script in parallel

if ddp:
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE']) # the total number of GPUs across all nodes

    device = f'cuda:{ddp_local_rank}' # Device name within that node. All nodes index GPUs as cuda:0, cuda:1, etc. Thus, we use local rank
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # if the zeroth GPU, then this will be true. For checkpointint, logging, etc.

else:
    # non ddp run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'